# Serving to the World

We already saw how to interact locally with http servers and how to tunnel a world-accessible URL back to your local server. But what if you want to set something up (e.g., in the cloud) and let others interact with the server, persistently?

## Python Serverless Microframework for AWS

https://github.com/awslabs/chalice

In [ ]:
# !pip install chalice

In [1]:
!chalice new-project helloworld

Directory already exists: helloworld
Aborted!


In [2]:
cd helloworld

/Users/jbloom/Classes/python-seminar/DataFiles_and_Notebooks/09_Web/helloworld


In [4]:
%%writefile app.py

from chalice import Chalice

app = Chalice(app_name="helloworld")

@app.route("/")
def index():
    return {"hello": "world, for real"}

Overwriting app.py


Deploy. You'll need to have set up your (free) [AWS credentials](https://aws.amazon.com/free/).

In [5]:
!AWS_DEFAULT_REGION=us-west-1 chalice deploy

Creating deployment package.
Creating IAM role: helloworld-dev
Creating lambda function: helloworld-dev
Creating Rest API
Resources deployed:
  - Lambda ARN: arn:aws:lambda:us-west-1:053013463530:function:helloworld-dev
  - Rest API URL: https://5ogwkenwug.execute-api.us-west-1.amazonaws.com/api/


In [6]:
!AWS_DEFAULT_REGION=us-west-1 chalice delete

Deleting Rest API: 5ogwkenwug
Deleting function: arn:aws:lambda:us-west-1:053013463530:function:helloworld-dev
Deleting IAM role: helloworld-dev


# Containerized Application

If you want to package your code AND other non-python files/executables, you need to ship a description of what it takes to get a system to a state with all the dependencies.

Docker is a way to specify the precise steps taken to install what you need to run your program. See http://containertutorials.com/index.html

In [7]:
!docker --help


Usage:	docker COMMAND

A self-sufficient runtime for containers

Options:
      --config string      Location of client config files (default
                           "/Users/jbloom/.docker")
  -D, --debug              Enable debug mode
  -H, --host list          Daemon socket(s) to connect to
  -l, --log-level string   Set the logging level
                           ("debug"|"info"|"warn"|"error"|"fatal")
                           (default "info")
      --tls                Use TLS; implied by --tlsverify
      --tlscacert string   Trust certs signed only by this CA (default
                           "/Users/jbloom/.docker/ca.pem")
      --tlscert string     Path to TLS certificate file (default
                           "/Users/jbloom/.docker/cert.pem")
      --tlskey string      Path to TLS key file (default
                           "/Users/jbloom/.docker/key.pem")
      --tlsverify          Use TLS and verify the remote
  -v, --version            Print version information 

In [ ]:
!git clone https://github.com/profjsb/pydocker.git

In [ ]:
cd pydocker/

In [ ]:
!ls

In [ ]:
!cat Dockerfile

In [ ]:
!cat app.py

In [ ]:
!cat Makefile

In [ ]:
!make build

In [ ]:
!make run

Now, check out http://localhost:5000

In [ ]:
!docker ps

In [ ]:
!docker logs mydocker-default

In [ ]:
!docker rm -f mydocker-default

In [ ]:
!docker login

In [ ]:
!docker push pyseminar/pydocker:latest

Now we can deploy this anywhere where Docker is run in the cloud. AWS Elastic Container Service makes this pretty easy: https://us-west-1.console.aws.amazon.com/ecs/home?region=us-west-1#/getStarted
